# Import libraries

In [1]:
import sys
from pathlib import Path
sys.path.insert(0,'..')
import glob
#!pip install openpyxl

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import h5py
import re

from functools import partial
from multiprocessing import Pool

from lhcsmapi.Time import Time
from lhcsmapi.metadata import signal_metadata
from lhcsmapi.pyedsl.dbsignal.post_mortem.PmDbRequest import PmDbRequest
from lhcsmapi.analysis.RbCircuitQuery import RbCircuitQuery

from src.acquisitions.current_voltage_diode_leads_nxcals import CurrentVoltageDiodeLeadsNXCALS
from src.acquisitions.current_voltage_diode_leads_pm import CurrentVoltageDiodeLeadsPM
from src.acquisitions.ee_t_res_pm import EETResPM
from src.acquisitions.ee_u_dump_res_pm import EEUDumpResPM
from src.acquisitions.leads import Leads
from src.acquisitions.pc_pm import PCPM
from src.acquisitions.qh_pm import QHPM
from src.acquisitions.voltage_logic_iqps import VoltageLogicIQPS
from src.acquisitions.voltage_nqps import VoltageNQPS
from src.acquisitions.voltage_nxcals import VoltageNXCALS

from src.utils.utils import log_acquisition
from src.utils.hdf_tools import acquisition_to_hdf5, load_from_hdf_with_regex
from src.utils.mp3_excel_processing import get_fgc_timestamp, get_fgc_timestamp_missing
from src.visualisation.visualisation import plot_hdf

`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
`np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: htt

# Select events to download

In [3]:
mp3_fpa_df = pd.read_csv("../data/RB_TC_extract_2021_11_22_processed.csv")
mp3_fpa_df_unique = mp3_fpa_df.drop_duplicates(subset=['timestamp_fgc', 'Circuit Name'])

In [4]:
lower_limit = Time.to_unix_timestamp('2021-01-01 00:00:00+01:00')
mp3_fpa_df_period = mp3_fpa_df_unique[mp3_fpa_df_unique['timestamp_fgc'] >= lower_limit].reset_index(drop=True) 
len(mp3_fpa_df_period)

561

In [5]:
signal_groups = [PCPM, VoltageNXCALS, VoltageNQPS, VoltageLogicIQPS, EEUDumpResPM, QHPM]
file_dir = Path('/eos/project/m/ml-for-alarm-system/private/RB_signals')

## Test query

In [6]:
signal_groups = [VoltageNXCALS, VoltageNQPS]
file_dir = Path('/eos/project/m/ml-for-alarm-system/private/RB_signals')

In [7]:
# quench with precursor
fpa_identifier = {'circuit_type': 'RB',
                 'circuit_name': 'RB.A78',
                 'timestamp_fgc': 1616962174400000000}

# udiode not available
fpa_identifier = {'circuit_type': 'RB',
                 'circuit_name': 'RB.A45',
                 'timestamp_fgc': 1619328389660000000}

In [ ]:
context_path = file_dir / "context_test1"

for signal_group in signal_groups:
    group = signal_group(**fpa_identifier, spark=spark)
    acquisition_to_hdf5(acquisition=group, 
                        file_dir=file_dir, 
                        context_dir_name="context_test1",
                        failed_queries_dir_name="failed_test1",
                        data_dir_name="data_test1")
        
log_acquisition(identifier=fpa_identifier, log_data={"download_complete": True}, log_path=context_path)

	Querying NXCALS signal(s) MB.A8R4:U_DIODE_RB, MB.B8R4:U_DIODE_RB, MB.A9R4:U_DIODE_RB, MB.B9R4:U_DIODE_RB, MB.A10R4:U_DIODE_RB... from 2021-04-25 07:25:39.660 to 2021-04-25 07:34:49.660
	Querying NXCALS signal(s) MB.A8R4:U_EARTH_RB, MB.B8R4:U_EARTH_RB, MB.A11R4:U_EARTH_RB, MB.B10R4:U_EARTH_RB, MB.B13R4:U_EARTH_RB... from 2021-04-25 07:25:39.660 to 2021-04-25 07:34:49.660


## Load data

In [ ]:
file_name = f"{fpa_identifier['circuit_type']}_{fpa_identifier['circuit_name']}_{fpa_identifier['timestamp_fgc']}.hdf5"
file_path = file_dir / Path('data_test1') / file_name

signals = ['I_MEAS','VoltageNQPS.*U_DIODE','VoltageNXCALS.*U_DIODE','I_EARTH_PCNT', 'IEARTH.I_EARTH','U_QS0','U_1','U_2','I_HDS', 'U_HDS','EEUDumpResPM']
len(signals)

In [ ]:
data = load_from_hdf_with_regex(file_path)

In [ ]:
plot_hdf(data, column_regex=['I_MEAS','VoltageNQPS.*U_DIODE','VoltageNXCALS.*U_DIODE','I_EARTH_PCNT', 'IEARTH.I_EARTH'])

# Test

In [ ]:
from src.utils.hdf_tools import df_to_hdf, append_or_overwrite_hdf_group

In [ ]:
df_to_hdf

df = pd.DataFrame({"test":[1,2,3],"test1":[6,2,6]})

df_to_hdf(file_path=Path("test.hdf5"), df=df, hdf_dir="test_group")

In [ ]:
data = load_from_hdf_with_regex(file_path=Path("test.hdf5"))
data

## Query all FGC events

In [ ]:
Time.to_unix_timestamp('2021-01-01 00:00:00+01:00')

In [ ]:
# Time.to_string(1514761200000000000), Time.to_unix_timestamp('2018-01-01 00:00:00+01:00')
mp3_fpa_df_period = mp3_fpa_df_unique[mp3_fpa_df_unique['timestamp_fgc'] >= 1609455600000000000].reset_index(drop=True) 
len(mp3_fpa_df_period)

signals = ['I_MEAS','VoltageNQPS.*U_DIODE','VoltageNXCALS.*U_DIODE','I_EARTH_PCNT', 'IEARTH.I_EARTH','U_QS0','U_1','U_2','I_HDS', 'U_HDS','EEUDumpResPM']

In [ ]:
for index, row in mp3_fpa_df_period.iterrows():
    
    fpa_identifier = {'circuit_type': row['Circuit Family'],
                      'circuit_name': row['Circuit Name'],
                      'timestamp_fgc': int(row['timestamp_fgc'])}
    file_name = f"{fpa_identifier['circuit_type']}_{fpa_identifier['circuit_name']}_{fpa_identifier['timestamp_fgc']}.hdf5"
    Path(file_dir / Path('20220624_plots')).mkdir(parents=True, exist_ok=True)
    
    if not os.path.isfile(str(file_dir / Path('20220624_plots') / f"{fpa_identifier['circuit_type']}_{fpa_identifier['circuit_name']}_{fpa_identifier['timestamp_fgc']}.png")):
        for signal_group in signal_groups:
            group = signal_group(**fpa_identifier, spark=spark)
            acquisition_to_hdf5(acquisition=group, 
                                file_dir=file_dir,
                                context_dir_name="20220624_context",
                                failed_queries_dir_name="20220624_failed",
                                data_dir_name="20220624_data")

        log_acquisition(identifier=fpa_identifier, log_data={"download_complete": True}, log_path=context_path)

        file_path = file_dir / Path('20220624_data') / file_name
        data = load_from_hdf_with_regex(file_path)
        fig = plt.figure(figsize=(20, 15))
        for i, s in enumerate(signals):
            fig.add_subplot(4, 3, i+1)
            n_signals = 0
            for df in data:
                if bool(re.search(s, df.columns.values[0])):
                    plt.plot(df.index.values, df.values)
                    n_signals += 1
            plt.title(f"{s} ({n_signals})")

        plt.savefig(str(file_dir / Path('20220624_plots') / f"{fpa_identifier['circuit_type']}_{fpa_identifier['circuit_name']}_{fpa_identifier['timestamp_fgc']}.png"))
        plt.close()
    else:
        print(f"done: {file_name}")

In [ ]:
    file_path = file_dir / Path('20220624_data') / file_name
    data = load_from_hdf_with_regex(file_path)
    fig = plt.figure(figsize=(20, 15))
    for i, s in enumerate(signals):
        fig.add_subplot(4, 3, i+1)
        n_signals = 0
        for df in data:
            if bool(re.search(s, df.columns.values[0])):
                plt.plot(df.index.values, df.values)
                n_signals += 1
        plt.title(f"{s} ({n_signals})")
        
    Path(file_dir / Path('20220624_plots')).mkdir(parents=True, exist_ok=True)
    plt.savefig(str(file_dir / Path('20220624_plots') / f"{fpa_identifier['circuit_type']}_{fpa_identifier['circuit_name']}_{fpa_identifier['timestamp_fgc']}.png"))